In [1]:
import sys
from pathlib import Path
import os 

current_path = Path.cwd()

# Go to top of the root and append
root = current_path.parents[1]
sys.path.append(str(root))

os.chdir(root)
print(root)

/home/gridsan/hzli/CV-Fairness


In [2]:
import pandas as pd
import math

In [4]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import openpyxl

Excel avec 
- Les 9 files de base (6 cbm, 3 excel) 
- Un file avec la différence d'accuracy entre male et female pour chacun des groupes: en row les 9 datasets, en ordonnée "différence entre male et female du groupe 1", "du groupe 2", "total"
- 3 file, un pour balanced, imbalanced 1, imbalanced 2 pour voir les différences entre les groupes

In [4]:
# Read files

path_dict = {
    'Baseline Balanced': 'results/mscoco/single_label/baseline/balanced/accuracies_by_group.csv',
    'Baseline Imbalanced 1': 'results/mscoco/single_label/baseline/imbalanced1/accuracies_by_group.csv',
    'Baseline Imbalanced 2': 'results/mscoco/single_label/baseline/imbalanced2/accuracies_by_group.csv',
    
    'CBM Dense Balanced': 'results/mscoco/single_label/CBM/balanced_dense/accuracies_by_group.csv',
    'CBM Dense Imbalanced 1': 'results/mscoco/single_label/CBM/imbalanced1_dense/accuracies_by_group.csv',
    'CBM Dense Imbalanced 2': 'results/mscoco/single_label/CBM/imbalanced2_dense/accuracies_by_group.csv',
    
    'CBM Sparse Balanced': 'results/mscoco/single_label/CBM/balanced_sparse/accuracies_by_group.csv',
    'CBM Sparse Imbalanced 1': 'results/mscoco/single_label/CBM/imbalanced1_sparse/accuracies_by_group.csv',
    'CBM Sparse Imbalanced 2': 'results/mscoco/single_label/CBM/imbalanced2_sparse/accuracies_by_group.csv',
    
    'CBM Dense Balanced Gender': 'results/mscoco/single_label/CBM/balanced_dense_gender/accuracies_by_group.csv',
    'CBM Dense Imbalanced 1 Gender': 'results/mscoco/single_label/CBM/imbalanced1_dense_gender/accuracies_by_group.csv',
    'CBM Dense Imbalanced 2 Gender': 'results/mscoco/single_label/CBM/imbalanced2_dense_gender/accuracies_by_group.csv',
    
    'CBM Sparse Balanced Gender': 'results/mscoco/single_label/CBM/balanced_sparse_gender/accuracies_by_group.csv',
    'CBM Sparse Imbalanced 1 Gender': 'results/mscoco/single_label/CBM/imbalanced1_sparse_gender/accuracies_by_group.csv',
    'CBM Sparse Imbalanced 2 Gender': 'results/mscoco/single_label/CBM/imbalanced2_sparse_gender/accuracies_by_group.csv',

}

In [5]:
result_dict = {}

for name, path in path_dict.items():
    result_dict[name] = pd.read_csv(path, index_col=0)

In [6]:
result_dict['Baseline Balanced']

,accuracy,correct,count
male_1,0.758621,462.0,609.0
male_2,0.796923,259.0,325.0
female_1,0.788177,480.0,609.0
female_2,0.827692,269.0,325.0


In [7]:
results_temp = []

for name_df,df in result_dict.items():
    if name_df in ['Baseline Imbalanced 1', 'Baseline Imbalanced 2', 'Baseline Balanced']:
            male_1, female_1, male_2, female_2 = df.loc['male_1', 'accuracy'], df.loc['female_1', 'accuracy'], df.loc['male_2', 'accuracy'], df.loc['female_2', 'accuracy']
    
            male_minus_female_1 = 100 * (male_1 - female_1)
            male_minus_female_2 = 100 * (male_2 - female_2)

            accuracy_parity_1 = abs(male_minus_female_1)
            accuracy_parity_2 = abs(male_minus_female_2)

            highest_accuracy_1 = 'male' if male_minus_female_1 > 0 else 'female'
            highest_accuracy_2 = 'male' if male_minus_female_2 > 0 else 'female'

            average_accuracy_1 = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'])
            average_accuracy_2 = 100 * (df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
            average_accuracy = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct'] + df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'] + df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
            
            result_temp_df = pd.DataFrame({'Experiment': [name_df], 
                              'Average Accuracy': [average_accuracy],
                              'Average Accuracy 1': [average_accuracy_1],
                              'Average Accuracy 2': [average_accuracy_2],
                              'Accuracy Parity 1': [accuracy_parity_1], 
                              'Accuracy Parity 2': [accuracy_parity_2],
                              'Highest accuracy 1': [highest_accuracy_1],
                              'Highest accuracy 2': [highest_accuracy_2],
                             })
            
            results_temp.append(result_temp_df)
            
final_temp_df = pd.concat(results_temp).set_index('Experiment')

In [8]:
final_temp_df

,Average Accuracy,Average Accuracy 1,Average Accuracy 2,Accuracy Parity 1,Accuracy Parity 2,Highest accuracy 1,Highest accuracy 2
Experiment,,,,,,,
Baseline Balanced,78.693790,77.339901,81.230769,2.955665,3.076923,female,female
Baseline Imbalanced 1,73.661670,72.988506,74.923077,4.105090,15.076923,male,female
Baseline Imbalanced 2,74.357602,73.070608,76.769231,10.509031,12.000000,female,male


In [9]:
final_temp_df.loc['Baseline Balanced', 'Average Accuracy']

78.69379014989293

In [10]:
results = []

for name, df in result_dict.items():
    male_1, female_1, male_2, female_2 = df.loc['male_1', 'accuracy'], df.loc['female_1', 'accuracy'], df.loc['male_2', 'accuracy'], df.loc['female_2', 'accuracy']
    
    male_minus_female_1 = 100 * (male_1 - female_1)
    male_minus_female_2 = 100 * (male_2 - female_2)
    
    accuracy_parity_1 = abs(male_minus_female_1)
    accuracy_parity_2 = abs(male_minus_female_2)
    
    highest_accuracy_1 = 'male' if male_minus_female_1 > 0 else 'female'
    highest_accuracy_2 = 'male' if male_minus_female_2 > 0 else 'female'
    
    average_accuracy_1 = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'])
    average_accuracy_2 = 100 * (df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
    average_accuracy = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct'] + df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'] + df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
    
    if '1' in name:
        percentage_accuracy = (average_accuracy - final_temp_df.loc['Baseline Imbalanced 1', 'Average Accuracy'])
        percentage_ap_1 = (accuracy_parity_1 - final_temp_df.loc['Baseline Imbalanced 1', 'Accuracy Parity 1'])
        percentage_ap_2 = (accuracy_parity_2 - final_temp_df.loc['Baseline Imbalanced 1', 'Accuracy Parity 2'])
        
    elif '2' in name:
        percentage_accuracy = (average_accuracy - final_temp_df.loc['Baseline Imbalanced 2', 'Average Accuracy']) 
        percentage_ap_1 = (accuracy_parity_1 - final_temp_df.loc['Baseline Imbalanced 2', 'Accuracy Parity 1']) 
        percentage_ap_2 = (accuracy_parity_2 - final_temp_df.loc['Baseline Imbalanced 2', 'Accuracy Parity 2'])
    
    else: 
        percentage_accuracy = (average_accuracy - final_temp_df.loc['Baseline Balanced', 'Average Accuracy']) 
        percentage_ap_1 = (accuracy_parity_1 - final_temp_df.loc['Baseline Balanced', 'Accuracy Parity 1']) 
        percentage_ap_2 = (accuracy_parity_2 - final_temp_df.loc['Baseline Balanced', 'Accuracy Parity 2']) 
    
    
    result_df = pd.DataFrame({'Experiment': [name], 
                              'Average Accuracy': [average_accuracy],
                              'Average Accuracy 1': [average_accuracy_1],
                              'Average Accuracy 2': [average_accuracy_2],
                              'Accuracy Parity 1': [accuracy_parity_1], 
                              'Accuracy Parity 2': [accuracy_parity_2],
                              'Highest accuracy 1': [highest_accuracy_1],
                              'Highest accuracy 2': [highest_accuracy_2],
                              'Accuracy change to Baseline': [percentage_accuracy],
                              'Accuracy Parity 1 change to Baseline': [percentage_ap_1],
                              'Accuracy Parity 2 change to Baseline': [percentage_ap_2],
                             })
    results.append(result_df)
    
final_df = pd.concat(results).reset_index(drop=True)

In [11]:
df_balanced = final_df[final_df['Experiment'].isin(['Baseline Balanced', 'CBM Dense Balanced', 'CBM Sparse Balanced', 'CBM Dense Balanced Gender', 'CBM Sparse Balanced Gender'])]
df_imbalanced_1 = final_df[final_df['Experiment'].isin(['Baseline Imbalanced 1', 'CBM Dense Imbalanced 1', 'CBM Sparse Imbalanced 1', 'CBM Dense Imbalanced 1 Gender', 'CBM Sparse Imbalanced 1 Gender'])]
df_imbalanced_2 = final_df[final_df['Experiment'].isin(['Baseline Imbalanced 2', 'CBM Dense Imbalanced 2', 'CBM Sparse Imbalanced 2', 'CBM Dense Imbalanced 2 Gender', 'CBM Sparse Imbalanced 2 Gender'])]

In [12]:
result_dict['Comparison Metrics'] = final_df
result_dict['Comparison Metrics Balanced'] = df_balanced
result_dict['Comparison Metrics Imbalanced 1'] = df_imbalanced_1
result_dict['Comparison Metrics Imbalanced 2'] = df_imbalanced_2

In [13]:
with pd.ExcelWriter(Path.cwd() / 'results' / 'mscoco' / 'single_label' / 'results.xlsx') as writer:
    for name, df in result_dict.items():
        df.to_excel(writer, sheet_name=name, index=False)

## Data Analysis

In [14]:
df = result_dict['Comparison Metrics']

In [15]:
df

,Experiment,Average Accuracy,Average Accuracy 1,Average Accuracy 2,Accuracy Parity 1,Accuracy Parity 2,Highest accuracy 1,Highest accuracy 2,Accuracy change to Baseline,Accuracy Parity 1 change to Baseline,Accuracy Parity 2 change to Baseline
0,Baseline Balanced,78.693790,77.339901,81.230769,2.955665,3.076923,female,female,0.000000,0.000000,0.000000
1,Baseline Imbalanced 1,73.661670,72.988506,74.923077,4.105090,15.076923,male,female,0.000000,0.000000,0.000000
2,Baseline Imbalanced 2,74.357602,73.070608,76.769231,10.509031,12.000000,female,male,0.000000,0.000000,0.000000
3,CBM Dense Balanced,76.713062,76.026273,78.000000,2.627258,4.000000,female,female,-1.980728,-0.328407,0.923077
4,CBM Dense Imbalanced 1,75.267666,74.958949,75.846154,6.732348,15.076923,male,female,1.605996,2.627258,0.000000
5,CBM Dense Imbalanced 2,74.732334,73.563218,76.923077,8.538588,5.538462,female,male,0.374732,-1.970443,-6.461538
6,CBM Sparse Balanced,73.822270,73.152709,75.076923,2.791461,4.307692,female,female,-4.871520,-0.164204,1.230769
7,CBM Sparse Imbalanced 1,73.929336,73.481117,74.769231,4.105090,10.461538,male,female,0.267666,0.000000,-4.615385
8,CBM Sparse Imbalanced 2,73.072805,72.167488,74.769231,8.702791,3.692308,female,male,-1.284797,-1.806240,-8.307692
9,CBM Dense Balanced Gender,76.713062,76.026273,78.000000,2.627258,4.000000,female,female,-1.980728,-0.328407,0.923077


In [16]:
metrics = ['Accuracy change to Baseline', 'Accuracy Parity 1 change to Baseline', 'Accuracy Parity 2 change to Baseline']

balanced = ['CBM Dense Balanced', 'CBM Sparse Balanced', 'CBM Dense Balanced Gender', 'CBM Sparse Balanced Gender']
imbalanced_1 = ['CBM Dense Imbalanced 1', 'CBM Sparse Imbalanced 1', 'CBM Dense Imbalanced 1 Gender', 'CBM Sparse Imbalanced 1 Gender']
imbalanced_2 = ['CBM Dense Imbalanced 2', 'CBM Sparse Imbalanced 2', 'CBM Dense Imbalanced 2 Gender', 'CBM Sparse Imbalanced 2 Gender']

In [17]:
df.index.values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [18]:
result = {}

for experiment in balanced:
    result_experiment = {'Experiment': experiment}
    for metric in metrics:
        baseline_value = df.loc['Baseline Balanced', metric]
        experiment_value = df.loc[experiment, metric]
        percentage_change = (baseline_value - experiment_value) / (baseline_value)
        result_experiment[experiment] 

KeyError: 'Baseline Balanced'